In [13]:
from __future__ import print_function
import numpy as np

from hyperopt import hp
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

import Augmentor
import json

from tuner import utils
import net

In [9]:
def data():
    train_dname = '../examples/dataset/brain/train'
    test_dname = '../examples/dataset/brain/validation'
    df = utils.df_fromdir(train_dname)
    df = utils.oversampling_df(df, 80)
    #x_train, y_train = load_data.load_fromdf(df, resize=96)
    x_train, y_train = load_data.load_fromdf(df, resize=96, rescale=1)
    df = utils.df_fromdir(test_dname)
    #x_test, y_test = load_data.load_fromdf(df)
    x_test, y_test = load_data.load_fromdf(df, resize=96, rescale=1)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

In [10]:
best_run, best_model = optim.minimize(
    model=net.aug, data=data, algo=tpe.suggest, max_evals=10, trials=Trials(),
    notebook_name='tune_augmentor')

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import hp
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers import BatchNormalization, Flatten, Dense
except:
    pass

try:
    from keras.layers import Input, Conv2D, Convolution2D
except:
    pass

try:
    from keras.layers import Activation, concatenate, Dropout
except:
    pass

try:
    from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
except:
    pass

try:
    from keras.models import Sequential, Model
e

Epoch 9/10
 - 0s - loss: 1.3465 - acc: 0.3281 - val_loss: 12.6887 - val_acc: 0.1750
Epoch 10/10
 - 0s - loss: 1.3609 - acc: 0.3250 - val_loss: 12.0886 - val_acc: 0.2500
Test accuracy: 0.25
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Operations: 1
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Epoch 1/10
 - 1s - loss: 1.4095 - acc: 0.2812 - val_loss: 12.1318 - val_acc: 0.2250
Epoch 2/10
 - 0s - loss: 1.3467 - acc: 0.3500 - val_loss: 14.3700 - val_acc: 0.0750
Epoch 3/10
 - 0s - loss: 1.3159 - acc: 0.3937 - val_loss: 11.6617 - val_acc: 0.2500
Epoch 4/10
 - 0s - loss: 1.2546 - acc: 0.4562 - val_loss: 14.0510 - val_acc: 0.1250
Epoch 5/10
 - 0s - loss: 1.1326 - acc: 0.5594 - val_loss: 12.8947 - val_acc: 0.2000
Epoch 6/10
 - 0s - loss: 1.0316 - acc: 0.6312 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 7/10
 - 0s - loss: 0.9644 - acc: 0.6125 - 

Epoch 6/10
 - 0s - loss: 1.0560 - acc: 0.5844 - val_loss: 14.9092 - val_acc: 0.0750
Epoch 7/10
 - 0s - loss: 0.9634 - acc: 0.6250 - val_loss: 12.4915 - val_acc: 0.2250
Epoch 8/10
 - 0s - loss: 0.8294 - acc: 0.7156 - val_loss: 12.4915 - val_acc: 0.2250
Epoch 9/10
 - 0s - loss: 0.8024 - acc: 0.6750 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 10/10
 - 0s - loss: 0.7481 - acc: 0.6656 - val_loss: 12.8945 - val_acc: 0.2000
Test accuracy: 0.2


In [11]:
with open('cond.json', 'w') as f:
    json.dump(best_run, f)

In [12]:
def gen_p():
    with open('cond.json', 'r') as f:
        conds = json.load(f)
    p = Augmentor.Pipeline()
    p.flip_left_right(probability=0.5)
    if conds['conditional']:
        p.crop_random(probability=1, percentage_area=0.8)
        p.resize(probability=1, width=96, height=96)
    if conds['conditional_1']:
        p.random_erasing(probability=0.5, rectangle_area=0.2)
    if conds['conditional_2']:
        p.shear(probability=0.3, max_shear_left=2, max_shear_right=2) 
    return p

In [6]:
def tune(x_train, y_train, x_test, y_test):
    
    def gen_p():
        with open('cond.json', 'r') as f:
            conds = json.load(f)
        p = Augmentor.Pipeline()
        p.flip_left_right(probability=0.5)
        if conds['conditional']:
            p.crop_random(probability=1, percentage_area=0.8)
            p.resize(probability=1, width=96, height=96)
        if conds['conditional_1']:
            p.random_erasing(probability=0.5, rectangle_area=0.2)
        if conds['conditional_2']:
            p.shear(probability=0.3, max_shear_left=2, max_shear_right=2) 
        return p
    n_out = y_train.shape[-1] 
    input_shape = (96, 96, 3)
    batch_size = 32
    epochs = 100
    steps_per_epoch = len(x_train) // batch_size
    lossfun='categorical_crossentropy'
    optimizer='Adam'
    metrics=['accuracy']
    g = gen_p()

    inputs = Input(shape=input_shape)
    x = inputs
    ch = {{choice([16, 32])}}
    x = Conv2D(ch, (3, 3))(x)
    x = Conv2D(ch, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ch = {{choice([32, 64])}}
    x = Conv2D(ch, (3, 3))(x)
    x = Conv2D(ch, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(n_out)(x)
    x = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(loss=lossfun, optimizer=optimizer, metrics=metrics)
    model.fit_generator(
        g,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        verbose=2,
        validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [7]:
best_run, best_model = optim.minimize(
    model=tune, data=data, algo=tpe.suggest, max_evals=10, trials=Trials(),
    notebook_name='tune_augmentor')

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import hp
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers import BatchNormalization, Flatten, Dense
except:
    pass

try:
    from keras.layers import Input, Conv2D, Convolution2D
except:
    pass

try:
    from keras.layers import Activation, concatenate, Dropout
except:
    pass

try:
    from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
except:
    pass

try:
    from keras.models import Sequential, Model
e

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/home/fytroo/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/fytroo/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fytroo/anaconda3/lib/python3.6/site-packages/keras/utils/data_utils.py", line 630, in data_generator_task
    generator_output = next(self._generator)
TypeError: 'Pipeline' object is not an iterator



StopIteration: 

In [ ]:
def p_fromconds(conds):
    p = Augmentor.Pipeline()
    p.flip_left_right(probability=0.5)
    if conds['conditional']:
        p.crop_random(probability=1, percentage_area=0.8)
        p.resize(probability=1, width=96, height=96)
    if conds['conditional_1']:
        p.random_erasing(probability=0.5, rectangle_area=0.2)
    if conds['conditional_2']:
        p.shear(probability=0.3, max_shear_left=2, max_shear_right=2) 
    return p

In [4]:
def fit_generator(lossfun='categorical_crossentropy',
                  optimizer=keras.optimizers.rmsprop(lr=0.005, decay=1e-6),
                  metrics=['accuracy'],
                  batch_size=32,
                  epochs=1,
                  steps_per_epochs=None,
                  augmentor_conds=None):
    def wrap(tuning_aug):
        def forward(x_train, y_train, x_test, y_test):
            n_out = y_train.shape[-1]
            input_shape = x_train.shape[1:]
            p = p_fromconds(best_run)
            g = p.keras_generator_from_array(x_train, y_train, batch_size=args.bs)
            g = ( (x/255., y) for (x,y) in g)
            
            model = target_net(n_out=n_out, input_shape=input_shape)
            print('build model')
            model.compile(loss=lossfun, optimizer=optimizer, metrics=metrics)
            model.fit_generator(
                g,
                batch_size=batch_size,
                epochs=epochs,
                steps_per_epochs=steps_per_epochs if steps_per_epochs else len(x_train)//batch_size,
                verbose=2,
                validation_data=(x_test, y_test))
            score, acc = model.evaluate(x_test, y_test, verbose=0)
            print('Test accuracy:', acc)
            return {'loss': -acc, 'status': STATUS_OK, 'model': model}
        return forward
    return wrap

In [12]:
model = fit_generator(augmentor_conds=best_run, epochs=100,
                     optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999))(net.tune)

In [15]:
@fit_generator(
    augmentor_conds=best_run, epochs=100,
    optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999))
def tune(x_train, y_train, x_test, y_test):
    n_out = len(x_train)
    input_shape = (96, 96, 3)
    batch_size = 32
    epochs = 200
    steps_per_epoch = 100

    inputs = Input(shape=input_shape)
    x = inputs
    ch = {{choice([16, 32])}}
    x = Conv2D(ch, (3, 3))(x)
    x = Conv2D(ch, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ch = {{choice([32, 64])}}
    x = Conv2D(ch, (3, 3))(x)
    x = Conv2D(ch, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(n_out)(x)
    x = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=x)
    return model

In [16]:
best_run, best_model = optim.minimize(
    model=tune, data=data, algo=tpe.suggest, max_evals=10, trials=Trials(),
    notebook_name='tune_augmentor')

IndentationError: unexpected indent (<unknown>, line 1)